## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-08-15-49-00 +0000,wsj,"Employment Survey Weakens, Adding to Signs of ...",https://www.wsj.com/economy/jobs/employment-su...
1,2025-09-08-15-48-33 +0000,nyt,DJI’s Chinese Drones Face U.S. Ban,https://www.nytimes.com/2025/09/08/business/dj...
2,2025-09-08-15-44-54 +0000,nyt,"In the Battle for Congress, Working-Class Demo...",https://www.nytimes.com/2025/09/07/us/politics...
3,2025-09-08-15-41-32 +0000,latimes,Baby snatched at knifepoint by father and gran...,https://www.latimes.com/california/story/2025-...
4,2025-09-08-15-38-00 +0000,wsj,At Least Six Killed in Jerusalem Shooting Attack,https://www.wsj.com/world/middle-east/jerusale...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
64,trump,25
203,new,12
151,raid,11
34,attack,10
333,city,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
21,2025-09-08-14-55-27 +0000,nypost,Trump demands foreign firms in US ‘hire and tr...,https://nypost.com/2025/09/08/business/trump-d...,64
72,2025-09-08-11-52-35 +0000,cbc,Israel vows 'hurricane' of strikes on Gaza to ...,https://www.cbc.ca/news/world/israel-gaza-city...,64
47,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...,61
46,2025-09-08-13-52-53 +0000,nyt,Trump Issues Warning to Hamas as Israeli Milit...,https://www.nytimes.com/2025/09/08/world/middl...,58
196,2025-09-07-22-34-59 +0000,latimes,"South Korea, irked at U.S. raid at Hyundai pla...",https://www.latimes.com/business/story/2025-09...,56


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
21,64,2025-09-08-14-55-27 +0000,nypost,Trump demands foreign firms in US ‘hire and tr...,https://nypost.com/2025/09/08/business/trump-d...
238,49,2025-09-07-17-38-00 +0000,wsj,Russia’s Largest Drone Attack Yet Hits Ukraine...,https://www.wsj.com/world/russias-largest-dron...
159,48,2025-09-08-03-34-58 +0000,nypost,Father on run with 3 kids in wilderness for 4 ...,https://nypost.com/2025/09/07/world-news/fugit...
73,43,2025-09-08-11-46-03 +0000,nypost,Israel threatens to destroy Gaza City in a ‘mi...,https://nypost.com/2025/09/08/world-news/israe...
160,42,2025-09-08-03-32-23 +0000,bbc,What it was like inside court as mushroom murd...,https://www.bbc.com/news/videos/c15k4y5nwz7o?a...
40,39,2025-09-08-14-06-44 +0000,nypost,Police in Nepal open fire on protesters outsid...,https://nypost.com/2025/09/08/world-news/polic...
196,36,2025-09-07-22-34-59 +0000,latimes,"South Korea, irked at U.S. raid at Hyundai pla...",https://www.latimes.com/business/story/2025-09...
47,33,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...
147,32,2025-09-08-05-17-27 +0000,nypost,Elderly California jewelry store owner assault...,https://nypost.com/2025/09/08/us-news/californ...
109,29,2025-09-08-09-30-00 +0000,wsj,The U.S. labor market would be barely growing ...,https://www.wsj.com/economy/jobs/healthcare-jo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
